In [1]:
import pandas as pd
import tensorflow as tf
import time
import numpy as np

In [76]:
data = pd.read_csv('./data.csv')

In [98]:
data_train = data[data.sentiment == 0][:100]

In [99]:
data_train = data_train.drop(['sentiment'], axis=1)

In [100]:
data_train.head()

,reviews
81970,Probably the best financial market course on c...
81971,"From a knowledge perspective, I have learned a..."
81972,Some parts of the course were informative. So...
81973,"The course is not that challenging, it could b..."
81974,It's more of a course for the people who have ...


In [101]:
x = []
sentences = list(data_train["reviews"])
for sen in sentences:
    x.append(sen)

In [102]:
import tensorflow as tf
vocab_size=10000
# Tokenize the text
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
max_length=100

# Pad the sequences to the same length
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")


In [103]:
padded_sequences

array([[ 80,   2, 179, ...,   0,   0,   0],
       [ 84,   4,  68, ...,   0,   0,   0],
       [ 37,  87,   3, ...,   0,   0,   0],
       ...,
       [  9,  42,   5, ...,   0,   0,   0],
       [  9, 272,   7, ...,   0,   0,   0],
       [ 40,  18, 129, ...,   0,   0,   0]])

In [104]:
# Define the generator model
def make_generator_model():
    embedding_dim=100
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(tf.keras.layers.Reshape((max_length, embedding_dim)))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation="softmax")))
    return model

# Define the discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential()
    embedding_dim=100
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(tf.keras.layers.Reshape((max_length, embedding_dim)))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation="sigmoid")))
    return model


# Instantiate the generator and discriminator models
generator = make_generator_model()
discriminator = make_discriminator_model()
# Define the loss functions and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
generator_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
batch_size=10
noise_dim=100

# Define the training loop
@tf.function
def train_step(real_sequences):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_sequences = generator(noise, training=True)

        real_loss = cross_entropy(tf.ones_like(discriminator(real_sequences, training=True)), discriminator(real_sequences, training=True))
        fake_loss = cross_entropy(tf.zeros_like(discriminator(generated_sequences, training=True)), discriminator(generated_sequences, training=True))
        disc_loss = real_loss + fake_loss

        gen_loss = generator_loss(tf.ones_like(discriminator(generated_sequences, training=True)), generated_sequences)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Define the training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for batch in dataset:
            train_step(batch)

        print(f"Time for epoch {epoch+1} is {time.time()-start} sec")


In [109]:
dataset = tf.data.Dataset.from_tensor_slices(padded_sequences)
dataset = dataset.shuffle(buffer_size=len(padded_sequences)).batch(100)
train(dataset, 10)
dataset

ValueError: in user code:

    File "<ipython-input-104-cb5b9f3bdc5c>", line 45, in train_step  *
        fake_loss = cross_entropy(tf.zeros_like(discriminator(generated_sequences, training=True)), discriminator(generated_sequences, training=True))
    File "C:\Users\HP\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "reshape_27" (type Reshape).
    
    Cannot reshape a tensor with 1000000000 elements to shape [10,100,100] (100000 elements) for '{{node sequential_40/reshape_27/Reshape_2}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32](sequential_40/embedding_39/embedding_lookup_2/Identity_1, sequential_40/reshape_27/Reshape_2/shape)' with input shapes: [10,100,10000,100], [3] and with input tensors computed as partial shapes: input[1] = [10,100,100].
    
    Call arguments received:
      • inputs=tf.Tensor(shape=(10, 100, 10000, 100), dty

In [107]:
len(padded_sequences)

100